# Twitter Scraper using Selenium

Scraper for Twitter Tweets using selenium. It can scrape tweets from:
- Home/New Feeds
- User Profile Tweets
- Query or Search Tweets
- Hashtags Tweets
- Advanced Search Tweets

In [10]:
import os
import sys
import pandas as pd

from datetime import datetime
from fake_headers import Headers
from time import sleep
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (
    NoSuchElementException,
    StaleElementReferenceException,
    WebDriverException,
)
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager

# Progress Class

Class for the progress of the scraper instance.

In [11]:
class Progress:
    def __init__(self, current, total) -> None:
        self.current = current
        self.total = total
        pass

    def print_progress(self, current) -> None:
        self.current = current
        progress = current / self.total
        bar_length = 40
        progress_bar = (
            "["
            + "=" * int(bar_length * progress)
            + "-" * (bar_length - int(bar_length * progress))
            + "]"
        )
        sys.stdout.write(
            "\rProgress: [{:<40}] {:.2%} {} of {}".format(
                progress_bar, progress, current, self.total
            )
        )
        sys.stdout.flush()


# Scroller Class

Class for the scrollbar of the web page.

In [12]:
class Scroller:
    def __init__(self, driver) -> None:
        self.driver = driver
        self.current_position = 0
        self.last_position = driver.execute_script("return window.pageYOffset;")
        self.scrolling = True
        self.scroll_count = 0
        pass

    def reset(self) -> None:
        self.current_position = 0
        self.last_position = self.driver.execute_script("return window.pageYOffset;")
        self.scroll_count = 0
        pass

    def scroll_to_top(self) -> None:
        self.driver.execute_script("window.scrollTo(0, 0);")
        pass

    def scroll_to_bottom(self) -> None:
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        pass

    def update_scroll_position(self) -> None:
        self.current_position = self.driver.execute_script("return window.pageYOffset;")
        pass


# Tweet Class

Object for the tweet. Including its data.

In [13]:
class Tweet:
    def __init__(
        self,
        card: WebDriver,
        driver: WebDriver,
        actions: ActionChains,
        scrape_poster_details=False
    ) -> None:
        self.card = card
        self.error = False
        self.tweet = None

        try:
            self.user = card.find_element(
                "xpath", './/div[@data-testid="User-Name"]//span'
            ).text
        except NoSuchElementException:
            self.error = True
            self.user = "skip"

        try:
            self.handle = card.find_element(
                "xpath", './/span[contains(text(), "@")]'
            ).text
        except NoSuchElementException:
            self.error = True
            self.handle = "skip"

        try:
            self.date_time = card.find_element("xpath", ".//time").get_attribute(
                "datetime"
            )

            if self.date_time is not None:
                self.is_ad = False
        except NoSuchElementException:
            self.is_ad = True
            self.error = True
            self.date_time = "skip"
        
        if self.error:
            return

        try:
            card.find_element(
                "xpath", './/*[local-name()="svg" and @data-testid="icon-verified"]'
            )

            self.verified = True
        except NoSuchElementException:
            self.verified = False

        self.content = ""
        contents = card.find_elements(
            "xpath",
            '(.//div[@data-testid="tweetText"])[1]/span | (.//div[@data-testid="tweetText"])[1]/a',
        )

        for index, content in enumerate(contents):
            self.content += content.text

        try:
            self.reply_cnt = card.find_element(
                "xpath", './/div[@data-testid="reply"]//span'
            ).text
            
            if self.reply_cnt == "":
                self.reply_cnt = "0"
        except NoSuchElementException:
            self.reply_cnt = "0"

        try:
            self.retweet_cnt = card.find_element(
                "xpath", './/div[@data-testid="retweet"]//span'
            ).text
            
            if self.retweet_cnt == "":
                self.retweet_cnt = "0"
        except NoSuchElementException:
            self.retweet_cnt = "0"

        try:
            self.like_cnt = card.find_element(
                "xpath", './/div[@data-testid="like"]//span'
            ).text
            
            if self.like_cnt == "":
                self.like_cnt = "0"
        except NoSuchElementException:
            self.like_cnt = "0"

        try:
            self.analytics_cnt = card.find_element(
                "xpath", './/a[contains(@href, "/analytics")]//span'
            ).text
            
            if self.analytics_cnt == "":
                self.analytics_cnt = "0"
        except NoSuchElementException:
            self.analytics_cnt = "0"

        try:
            self.tags = card.find_elements(
                "xpath",
                './/a[contains(@href, "src=hashtag_click")]',
            )

            self.tags = [tag.text for tag in self.tags]
        except NoSuchElementException:
            self.tags = []
        
        try:
            self.mentions = card.find_elements(
                "xpath",
                '(.//div[@data-testid="tweetText"])[1]//a[contains(text(), "@")]',
            )

            self.mentions = [mention.text for mention in self.mentions]
        except NoSuchElementException:
            self.mentions = []
        
        try:
            raw_emojis = card.find_elements(
                "xpath",
                '(.//div[@data-testid="tweetText"])[1]/img[contains(@src, "emoji")]',
            )
            
            self.emojis = [emoji.get_attribute("alt").encode("unicode-escape").decode("ASCII") for emoji in raw_emojis]
        except NoSuchElementException:
            self.emojis = []
        
        try:
            self.profile_img = card.find_element(
                "xpath", './/div[@data-testid="Tweet-User-Avatar"]//img'
            ).get_attribute("src")
        except NoSuchElementException:
            self.profile_img = ""
            
        try:
            self.tweet_link = self.card.find_element(
                "xpath",
                ".//a[contains(@href, '/status/')]",
            ).get_attribute("href")
            self.tweet_id = str(self.tweet_link.split("/")[-1])
        except NoSuchElementException:
            self.tweet_link = ""
            self.tweet_id = ""
        
        self.following_cnt = "0"
        self.followers_cnt = "0"
        self.user_id = None
        
        if scrape_poster_details:
            el_name = card.find_element(
                "xpath", './/div[@data-testid="User-Name"]//span'
            )
            
            ext_hover_card = False
            ext_user_id = False
            ext_following = False
            ext_followers = False
            hover_attempt = 0
            
            while not ext_hover_card or not ext_user_id or not ext_following or not ext_followers:
                try:
                    actions.move_to_element(el_name).perform()
                    
                    hover_card = driver.find_element(
                        "xpath",
                        '//div[@data-testid="hoverCardParent"]'
                    )
                    
                    ext_hover_card = True
                    
                    while not ext_user_id:
                        try:
                            raw_user_id = hover_card.find_element(
                                "xpath",
                                '(.//div[contains(@data-testid, "-follow")]) | (.//div[contains(@data-testid, "-unfollow")])'
                            ).get_attribute("data-testid")
                            
                            if raw_user_id == "":
                                self.user_id = None
                            else:
                                self.user_id = str(raw_user_id.split("-")[0])
                            
                            ext_user_id = True
                        except NoSuchElementException:
                            continue
                        except StaleElementReferenceException:
                            self.error = True
                            return
                    
                    while not ext_following:
                        try:
                            self.following_cnt = hover_card.find_element(
                                "xpath",
                                './/a[contains(@href, "/following")]//span'
                            ).text
                            
                            if self.following_cnt == "":
                                self.following_cnt = "0"
                                
                            ext_following = True
                        except NoSuchElementException:
                            continue
                        except StaleElementReferenceException:
                            self.error = True
                            return
                    
                    while not ext_followers:
                        try:
                            self.followers_cnt = hover_card.find_element(
                                "xpath",
                                './/a[contains(@href, "/verified_followers")]//span'
                            ).text
                            
                            if self.followers_cnt == "":
                                self.followers_cnt = "0"
                            
                            ext_followers = True
                        except NoSuchElementException:
                            continue
                        except StaleElementReferenceException:
                            self.error = True
                            return
                except NoSuchElementException:
                    if hover_attempt==3:
                        self.error
                        return
                    hover_attempt+=1
                    sleep(0.5)
                    continue
                except StaleElementReferenceException:
                    self.error = True
                    return
            
            if ext_hover_card and ext_following and ext_followers:
                actions.reset_actions()
        
        self.tweet = (
            self.user,
            self.handle,
            self.date_time,
            self.verified,
            self.content,
            self.reply_cnt,
            self.retweet_cnt,
            self.like_cnt,
            self.analytics_cnt,
            self.tags,
            self.mentions,
            self.emojis,
            self.profile_img,
            self.tweet_link,
            self.tweet_id,
            self.user_id,
            self.following_cnt,
            self.followers_cnt,
        )

        pass


# Twitter Scraper Class

Class for the Twitter Scraper.

In [14]:
TWITTER_LOGIN_URL = "https://twitter.com/i/flow/login"

class Twitter_Scraper:
    def __init__(
        self,
        username,
        password,
        max_tweets=50,
        scrape_username=None,
        scrape_hashtag=None,
        scrape_query=None,
        scrape_poster_details=False,
        scrape_latest=True,
        scrape_top=False,
    ):
        print("Initializing Twitter Scraper...")
        self.username = username
        self.password = password
        self.interrupted = False
        self.tweet_ids = set()
        self.data = []
        self.tweet_cards = []
        self.scraper_details = {
            "type": None,
            "username": None,
            "hashtag": None,
            "query": None,
            "tab": None,
            "poster_details": False,
        }
        self.max_tweets = max_tweets
        self.progress = Progress(0, max_tweets)
        self.router = self.go_to_home
        self.driver = self._get_driver()
        self.actions = ActionChains(self.driver)
        self.scroller = Scroller(self.driver)
        self._config_scraper(
            max_tweets,
            scrape_username,
            scrape_hashtag,
            scrape_query,
            scrape_latest,
            scrape_top,
            scrape_poster_details,
        )

    def _config_scraper(
        self,
        max_tweets=50,
        scrape_username=None,
        scrape_hashtag=None,
        scrape_query=None,
        scrape_latest=True,
        scrape_top=False,
        scrape_poster_details=False,
    ):
        self.tweet_ids = set()
        self.data = []
        self.tweet_cards = []
        self.max_tweets = max_tweets
        self.progress = Progress(0, max_tweets)
        self.scraper_details = {
            "type": None,
            "username": scrape_username,
            "hashtag": str(scrape_hashtag).replace("#", "")
            if scrape_hashtag is not None
            else None,
            "query": scrape_query,
            "tab": "Latest" if scrape_latest else "Top" if scrape_top else "Latest",
            "poster_details": scrape_poster_details,
        }
        self.router = self.go_to_home
        self.scroller = Scroller(self.driver)

        if scrape_username is not None:
            self.scraper_details["type"] = "Username"
            self.router = self.go_to_profile
        elif scrape_hashtag is not None:
            self.scraper_details["type"] = "Hashtag"
            self.router = self.go_to_hashtag
        elif scrape_query is not None:
            self.scraper_details["type"] = "Query"
            self.router = self.go_to_search
        else:
            self.scraper_details["type"] = "Home"
            self.router = self.go_to_home
        pass

    def _get_driver(self):
        print("Setup WebDriver...")
        header = Headers().generate()["User-Agent"]

        browser_option = ChromeOptions()
        browser_option.add_argument("--no-sandbox")
        browser_option.add_argument("--disable-dev-shm-usage")
        browser_option.add_argument("--ignore-certificate-errors")
        browser_option.add_argument("--disable-gpu")
        browser_option.add_argument("--log-level=3")
        browser_option.add_argument("--disable-notifications")
        browser_option.add_argument("--disable-popup-blocking")
        browser_option.add_argument("--user-agent={}".format(header))

        # For Hiding Browser
        browser_option.add_argument("--headless")

        try:
            print("Initializing ChromeDriver...")
            driver = webdriver.Chrome(
                options=browser_option,
            )

            print("WebDriver Setup Complete")
            return driver
        except WebDriverException:
            try:
                print("Downloading ChromeDriver...")
                chromedriver_path = ChromeDriverManager().install()
                chrome_service = ChromeService(executable_path=chromedriver_path)

                print("Initializing ChromeDriver...")
                driver = webdriver.Chrome(
                    service=chrome_service,
                    options=browser_option,
                )

                print("WebDriver Setup Complete")
                return driver
            except Exception as e:
                print(f"Error setting up WebDriver: {e}")
                sys.exit(1)
        pass

    def login(self):
        print()
        print("Logging in to Twitter...")

        try:
            self.driver.maximize_window()
            self.driver.get(TWITTER_LOGIN_URL)
            sleep(3)

            self._input_username()
            self._input_unusual_activity()
            self._input_password()

            cookies = self.driver.get_cookies()

            auth_token = None

            for cookie in cookies:
                if cookie["name"] == "auth_token":
                    auth_token = cookie["value"]
                    break

            if auth_token is None:
                raise ValueError(
                    """This may be due to the following:

- Internet connection is unstable
- Username is incorrect
- Password is incorrect
"""
                )

            print()
            print("Login Successful")
            print()
        except Exception as e:
            print()
            print(f"Login Failed: {e}")
            sys.exit(1)

        pass

    def _input_username(self):
        input_attempt = 0

        while True:
            try:
                username = self.driver.find_element(
                    "xpath", "//input[@autocomplete='username']"
                )

                username.send_keys(self.username)
                username.send_keys(Keys.RETURN)
                sleep(3)
                break
            except NoSuchElementException:
                input_attempt += 1
                if input_attempt >= 3:
                    print()
                    print(
                        """There was an error inputting the username.

It may be due to the following:
- Internet connection is unstable
- Username is incorrect
- Twitter is experiencing unusual activity"""
                    )
                    self.driver.quit()
                    sys.exit(1)
                else:
                    print("Re-attempting to input username...")
                    sleep(2)

    def _input_unusual_activity(self):
        input_attempt = 0

        while True:
            try:
                unusual_activity = self.driver.find_element(
                    "xpath", "//input[@data-testid='ocfEnterTextTextInput']"
                )
                unusual_activity.send_keys(self.username)
                unusual_activity.send_keys(Keys.RETURN)
                sleep(3)
                break
            except NoSuchElementException:
                input_attempt += 1
                if input_attempt >= 3:
                    break

    def _input_password(self):
        input_attempt = 0

        while True:
            try:
                password = self.driver.find_element(
                    "xpath", "//input[@autocomplete='current-password']"
                )

                password.send_keys(self.password)
                password.send_keys(Keys.RETURN)
                sleep(3)
                break
            except NoSuchElementException:
                input_attempt += 1
                if input_attempt >= 3:
                    print()
                    print(
                        """There was an error inputting the password.

It may be due to the following:
- Internet connection is unstable
- Password is incorrect
- Twitter is experiencing unusual activity"""
                    )
                    self.driver.quit()
                    sys.exit(1)
                else:
                    print("Re-attempting to input password...")
                    sleep(2)

    def go_to_home(self):
        self.driver.get("https://twitter.com/home")
        sleep(3)
        pass

    def go_to_profile(self):
        if (
            self.scraper_details["username"] is None
            or self.scraper_details["username"] == ""
        ):
            print("Username is not set.")
            sys.exit(1)
        else:
            self.driver.get(f"https://twitter.com/{self.scraper_details['username']}")
            sleep(3)
        pass

    def go_to_hashtag(self):
        if (
            self.scraper_details["hashtag"] is None
            or self.scraper_details["hashtag"] == ""
        ):
            print("Hashtag is not set.")
            sys.exit(1)
        else:
            url = f"https://twitter.com/hashtag/{self.scraper_details['hashtag']}?src=hashtag_click"
            if self.scraper_details["tab"] == "Latest":
                url += "&f=live"

            self.driver.get(url)
            sleep(3)
        pass

    def go_to_search(self):
        if self.scraper_details["query"] is None or self.scraper_details["query"] == "":
            print("Query is not set.")
            sys.exit(1)
        else:
            url = f"https://twitter.com/search?q={self.scraper_details['query']}&src=typed_query"
            if self.scraper_details["tab"] == "Latest":
                url += "&f=live"

            self.driver.get(url)
            sleep(3)
        pass

    def get_tweet_cards(self):
        self.tweet_cards = self.driver.find_elements(
            "xpath", '//article[@data-testid="tweet" and not(@disabled)]'
        )
        pass

    def remove_hidden_cards(self):
        try:
            hidden_cards = self.driver.find_elements(
                "xpath", '//article[@data-testid="tweet" and @disabled]'
            )

            for card in hidden_cards[1:-2]:
                self.driver.execute_script(
                    "arguments[0].parentNode.parentNode.parentNode.remove();", card
                )
        except Exception as e:
            return
        pass

    def scrape_tweets(
        self,
        max_tweets=50,
        scrape_username=None,
        scrape_hashtag=None,
        scrape_query=None,
        scrape_latest=True,
        scrape_top=False,
        scrape_poster_details=False,
        router=None,
    ):
        self._config_scraper(
            max_tweets,
            scrape_username,
            scrape_hashtag,
            scrape_query,
            scrape_latest,
            scrape_top,
            scrape_poster_details,
        )

        if router is None:
            router = self.router

        router()

        if self.scraper_details["type"] == "Username":
            print(
                "Scraping Tweets from @{}...".format(self.scraper_details["username"])
            )
        elif self.scraper_details["type"] == "Hashtag":
            print(
                "Scraping {} Tweets from #{}...".format(
                    self.scraper_details["tab"], self.scraper_details["hashtag"]
                )
            )
        elif self.scraper_details["type"] == "Query":
            print(
                "Scraping {} Tweets from {} search...".format(
                    self.scraper_details["tab"], self.scraper_details["query"]
                )
            )
        elif self.scraper_details["type"] == "Home":
            print("Scraping Tweets from Home...")

        self.progress.print_progress(0)

        refresh_count = 0
        added_tweets = 0
        empty_count = 0

        while self.scroller.scrolling:
            try:
                self.get_tweet_cards()
                added_tweets = 0

                for card in self.tweet_cards[-15:]:
                    try:
                        tweet_id = str(card)

                        if tweet_id not in self.tweet_ids:
                            self.tweet_ids.add(tweet_id)

                            if not self.scraper_details["poster_details"]:
                                self.driver.execute_script(
                                    "arguments[0].scrollIntoView();", card
                                )

                            tweet = Tweet(
                                card=card,
                                driver=self.driver,
                                actions=self.actions,
                                scrape_poster_details=self.scraper_details[
                                    "poster_details"
                                ],
                            )

                            if tweet:
                                if not tweet.error and tweet.tweet is not None:
                                    if not tweet.is_ad:
                                        self.data.append(tweet.tweet)
                                        added_tweets += 1
                                        self.progress.print_progress(len(self.data))

                                        if len(self.data) >= self.max_tweets:
                                            self.scroller.scrolling = False
                                            break
                                    else:
                                        continue
                                else:
                                    continue
                            else:
                                continue
                        else:
                            continue
                    except NoSuchElementException:
                        continue

                if len(self.data) >= self.max_tweets:
                    break

                if added_tweets == 0:
                    if empty_count >= 5:
                        if refresh_count >= 3:
                            print()
                            print("No more tweets to scrape")
                            break
                        refresh_count += 1
                    empty_count += 1
                    sleep(1)
                else:
                    empty_count = 0
                    refresh_count = 0
            except StaleElementReferenceException:
                sleep(2)
                continue
            except KeyboardInterrupt:
                print("\n")
                print("Keyboard Interrupt")
                self.interrupted = True
                break
            except Exception as e:
                print("\n")
                print(f"Error scraping tweets: {e}")
                break

        print("")

        if len(self.data) >= self.max_tweets:
            print("Scraping Complete")
        else:
            print("Scraping Incomplete")

        print("Tweets: {} out of {}\n".format(len(self.data), self.max_tweets))

        pass

    def save_to_csv(self, scraped_username):
        print("Saving Tweets to CSV...")
        now = datetime.now()
        folder_path = "./tweets/"

        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print("Created Folder: {}".format(folder_path))

        data = {
            "Name": [tweet[0] for tweet in self.data],
            "Handle": [tweet[1] for tweet in self.data],
            "Timestamp": [tweet[2] for tweet in self.data],
            "Verified": [tweet[3] for tweet in self.data],
            "Content": [tweet[4] for tweet in self.data],
            "Comments": [tweet[5] for tweet in self.data],
            "Retweets": [tweet[6] for tweet in self.data],
            "Likes": [tweet[7] for tweet in self.data],
            "Analytics": [tweet[8] for tweet in self.data],
            "Tags": [tweet[9] for tweet in self.data],
            "Mentions": [tweet[10] for tweet in self.data],
            "Emojis": [tweet[11] for tweet in self.data],
            "Profile Image": [tweet[12] for tweet in self.data],
            "Tweet Link": [tweet[13] for tweet in self.data],
            "Tweet ID": [f'tweet_id:{tweet[14]}' for tweet in self.data],
        }

        if self.scraper_details["poster_details"]:
            data["Tweeter ID"] = [f'user_id:{tweet[15]}' for tweet in self.data]
            data["Following"] = [tweet[16] for tweet in self.data]
            data["Followers"] = [tweet[17] for tweet in self.data]

        df = pd.DataFrame(data)

        current_time = now.strftime("%Y-%m-%d_%H-%M-%S")
        file_path = f"tweets/{scraped_username}.csv"
        pd.set_option("display.max_colwidth", None)
        df.to_csv(file_path, index=False, encoding="utf-8")

        print("CSV Saved: {}".format(file_path))

        pass

    def get_tweets(self):
        return self.data

# Create a new instance of the Twitter Scraper class

In [15]:
USER_UNAME = os.environ['TWITTER_USERNAME']
USER_PASSWORD = os.environ['TWITTER_PASSWORD']

scraper = Twitter_Scraper(
    username='',
    password='',
    # max_tweets=10,
    # scrape_username="something",
    # scrape_hashtag="something",
    # scrape_query="something",
    # scrape_latest=False,
    # scrape_top=True,
    # scrape_poster_details=True
)

Initializing Twitter Scraper...
Setup WebDriver...
Initializing ChromeDriver...
WebDriver Setup Complete


In [16]:
scraper.login()


Logging in to Twitter...

Login Successful



# Run Twitter Scraper

In [7]:
import pandas as pd

data = {
    'Kurum': ['TBMM', 'Numan Kurtulmuş', 'TC Cumhurbaşkanlığı', 'RTE', 'Cevdet Yılmaz', 'İbrahim YUMAKLI', 'Yılmaz Tunç', 'Hakan FİDAN', 'Alparslan BAYRAKTAR', 'Mehmet ŞİMŞEK'
              , 'Vedat IŞIKHAN', 'Mehmet ÖZHASEKİ', 'Abdulkadir URALOĞLU', 'Mehmet Nuri ERSOY', 'Mahinur ÖZDEMİR GÖKTAŞ', 'Osman Aşkın BAK', 
              'Ali YERLİKAYA', 'Yusuf TEKİN',  'Fahrettin KOCA', 'Mehmet Fatih KACIR', 'Ömer BOLAT', 'Hafize Gaye Erkan', 'CB Yardımcısı'],
    'Twitter_Link': ['https://twitter.com/tbmmresmi', 'https://twitter.com/NumanKurtulmus', 'https://twitter.com/tcbestepe', 
                     'https://twitter.com/RTErdogan', 'https://twitter.com/_cevdetyilmaz', 'https://twitter.com/ibrahimyumakli',
                     'https://twitter.com/yilmaztunc', 'https://twitter.com/HakanFidan', 'https://twitter.com/aBayraktar1', 'https://twitter.com/memetsimsek', 
                     'https://twitter.com/isikhanvedat', 'https://twitter.com/mehmetozhaseki', 'https://twitter.com/a_uraloglu', 'https://twitter.com/MehmetNuriErsoy',
                     'https://twitter.com/MahinurOzdemir', 'https://twitter.com/OA_BAK', 'https://twitter.com/TC_icisleri', 'https://twitter.com/Yusuf__Tekin', 
                     'https://twitter.com/drfahrettinkoca', 'https://twitter.com/mfatihkacir', 'https://twitter.com/omerbolatTR', 'https://twitter.com/hafizegayeerkan', 'https://twitter.com/_cevdetyilmaz']
}

data2 = {
    'Kurum': [
        'TBMM',
        'TC Cumhurbaşkanlığı',
        'TC Cumhurbaşkanlığı CB Yardımcısı',
        'TC Tarım ve Orman Bakanlığı',
        'TC Adalet Bakanlığı',
        'TC Dış İşleri Bakanlığı',
        'TC Enerji ve Tabii Kaynak. Bak.',
        'TC Hazine ve Maliye Bakanlığı',
        'TC Çalışma ve Sosyal Güv. Bak.',
        'TC Çevre, Şehir, İklim Bak.',
        'TC Ulaştırma ve Altyapı Bak.',
        'TC Kültür ve Turizm Bakanlığı',
        'TC Aile ve Sosyal Hizmetler Bak.',
        'TC Gençlik ve Spor Bak.',
        'TC İç İşleri Bak.',
        'TC Milli Eğitim Bakanlığı',
        'TC Milli Savunma Bakanlığı',
        'TC Sağlık Bakanlığı',
        'TC Sanayi ve Teknoloji Bakanlığı',
        'TC Ticaret Bakanlığı',
        'TC Merkez Bankası'
    ],
    'Twitter_Link': [
        'https://twitter.com/tbmmresmi',
        'https://twitter.com/tcbestepe',
        'https://twitter.com/adalet_bakanlik',
        'https://twitter.com/TCTarim',
        'https://twitter.com/adalet_bakanlik',
        'https://twitter.com/TC_Disisleri',
        'https://twitter.com/TCEnerji',
        'https://twitter.com/HMBakanligi',
        'https://twitter.com/csgbakanligi',
        'https://twitter.com/csbgovtr',
        'https://twitter.com/UABakanligi',
        'https://twitter.com/TCKulturTurizm',
        'https://twitter.com/tcailesosyal',
        'https://twitter.com/gencliksporbak',
        'https://twitter.com/TC_icisleri',
        'https://twitter.com/tcmeb',
        'https://twitter.com/tcsavunma',
        'https://twitter.com/saglikbakanligi',
        'https://twitter.com/TCSanayi',
        'https://twitter.com/ticaret',
        'https://twitter.com/Merkez_Bankasi'
    ]
}

df1 = pd.DataFrame(data)
df2 = pd.DataFrame(data2)

df = pd.concat([df1, df2], ignore_index=True).drop_duplicates(subset=['Kurum']).reset_index(drop=True)

df

,Kurum,Twitter_Link
0,TBMM,https://twitter.com/tbmmresmi
1,Numan Kurtulmuş,https://twitter.com/NumanKurtulmus
2,TC Cumhurbaşkanlığı,https://twitter.com/tcbestepe
3,RTE,https://twitter.com/RTErdogan
4,Cevdet Yılmaz,https://twitter.com/_cevdetyilmaz
5,İbrahim YUMAKLI,https://twitter.com/ibrahimyumakli
6,Yılmaz Tunç,https://twitter.com/yilmaztunc
7,Hakan FİDAN,https://twitter.com/HakanFidan
8,Alparslan BAYRAKTAR,https://twitter.com/aBayraktar1
9,Mehmet ŞİMŞEK,https://twitter.com/memetsimsek


In [8]:
twitter_handles = ["@" + handle for handle in df['Twitter_Link'].str.split('/').str[-1].tolist()]
twitter_handles

['@tbmmresmi',
 '@NumanKurtulmus',
 '@tcbestepe',
 '@RTErdogan',
 '@_cevdetyilmaz',
 '@ibrahimyumakli',
 '@yilmaztunc',
 '@HakanFidan',
 '@aBayraktar1',
 '@memetsimsek',
 '@isikhanvedat',
 '@mehmetozhaseki',
 '@a_uraloglu',
 '@MehmetNuriErsoy',
 '@MahinurOzdemir',
 '@OA_BAK',
 '@TC_icisleri',
 '@Yusuf__Tekin',
 '@drfahrettinkoca',
 '@mfatihkacir',
 '@omerbolatTR',
 '@hafizegayeerkan',
 '@_cevdetyilmaz',
 '@adalet_bakanlik',
 '@TCTarim',
 '@adalet_bakanlik',
 '@TC_Disisleri',
 '@TCEnerji',
 '@HMBakanligi',
 '@csgbakanligi',
 '@csbgovtr',
 '@UABakanligi',
 '@TCKulturTurizm',
 '@tcailesosyal',
 '@gencliksporbak',
 '@TC_icisleri',
 '@tcmeb',
 '@tcsavunma',
 '@saglikbakanligi',
 '@TCSanayi',
 '@ticaret',
 '@Merkez_Bankasi']

In [17]:
for user in twitter_handles:
    scraper.scrape_tweets(
        max_tweets=10,
        scrape_username=f"{user}",
        # scrape_hashtag="something",
        # scrape_query="something",
        # scrape_latest=False,
        # scrape_top=True,
        # scrape_poster_details=True,
    )
    scraper.save_to_csv(user.replace('@', ''))

Scraping Tweets from @@tbmmresmi...
Progress: [[============================------------]] 70.00% 7 of 10
No more tweets to scrape

Scraping Incomplete
Tweets: 7 out of 10

Saving Tweets to CSV...
CSV Saved: tweets/tbmmresmi.csv
Scraping Tweets from @@NumanKurtulmus...
Progress: [[========================================]] 100.00% 10 of 10
Scraping Complete
Tweets: 10 out of 10

Saving Tweets to CSV...
CSV Saved: tweets/NumanKurtulmus.csv
Scraping Tweets from @@tcbestepe...
Progress: [[========================================]] 100.00% 10 of 10
Scraping Complete
Tweets: 10 out of 10

Saving Tweets to CSV...
CSV Saved: tweets/tcbestepe.csv
Scraping Tweets from @@RTErdogan...
Progress: [[========================================]] 100.00% 10 of 10
Scraping Complete
Tweets: 10 out of 10

Saving Tweets to CSV...
CSV Saved: tweets/RTErdogan.csv
Scraping Tweets from @@_cevdetyilmaz...
Progress: [[========================================]] 100.00% 10 of 10
Scraping Complete
Tweets: 10 out of 1

In [18]:
import os
import pandas as pd

folder_path = 'tweets/'
folder_name = os.listdir(folder_path)

datasets = []
for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        df = pd.read_csv(folder_path + file)
        print(f'df handle: {df["Handle"].unique()} | folder name: {file.split(".")[0]}')
        df = df[df['Handle'] == f'@{file.split(".")[0]}']
        datasets.append(df)

combined_dataset = pd.concat(datasets)


df handle: ['@aBayraktar1' '@TCEnerji' '@RTErdogan'] | folder name: aBayraktar1
df handle: ['@adalet_bakanlik' '@yilmaztunc'] | folder name: adalet_bakanlik
df handle: ['@a_uraloglu'] | folder name: a_uraloglu
df handle: ['@mehmetozhaseki' '@csbgovtr'] | folder name: csbgovtr
df handle: ['@csgbakanligi'] | folder name: csgbakanligi
df handle: ['@drfahrettinkoca'] | folder name: drfahrettinkoca
df handle: ['@OA_BAK' '@gencliksporbak' '@Olympics'] | folder name: gencliksporbak
df handle: ['@hafizegayeerkan'] | folder name: hafizegayeerkan
df handle: ['@HakanFidan' '@RTErdogan'] | folder name: HakanFidan
df handle: ['@memetsimsek' '@HMBakanligi'] | folder name: HMBakanligi
df handle: ['@ibrahimyumakli' '@RTErdogan'] | folder name: ibrahimyumakli
df handle: ['@isikhanvedat' '@csgbakanligi' '@RTErdogan'] | folder name: isikhanvedat
df handle: ['@MahinurOzdemir' '@RTErdogan'] | folder name: MahinurOzdemir
df handle: ['@MehmetNuriErsoy' '@RTErdogan' '@tv100'] | folder name: MehmetNuriErsoy
df

In [19]:
combined_dataset['Handle'].value_counts()

Handle
@RTErdogan          10
@drfahrettinkoca    10
@Merkez_Bankasi     10
@mehmetozhaseki     10
@NumanKurtulmus     10
@csgbakanligi       10
@a_uraloglu         10
@yilmaztunc          9
@tcmeb               9
@ibrahimyumakli      9
@hafizegayeerkan     8
@HakanFidan          8
@tcbestepe           8
@MehmetNuriErsoy     8
@Yusuf__Tekin        8
@tcsavunma           8
@MahinurOzdemir      7
@mfatihkacir         7
@_cevdetyilmaz       7
@isikhanvedat        6
@TCSanayi            6
@TC_Disisleri        6
@TCKulturTurizm      6
@aBayraktar1         5
@omerbolatTR         5
@adalet_bakanlik     5
@tcailesosyal        4
@memetsimsek         4
@TCEnerji            3
@UABakanligi         3
@gencliksporbak      3
@TC_icisleri         2
@OA_BAK              2
@saglikbakanligi     1
@TCTarim             1
@HMBakanligi         1
@ticaret             1
@csbgovtr            1
Name: count, dtype: int64

In [20]:
combined_dataset.drop(columns=['Verified', 'Analytics', 'Tags', 'Mentions', 'Emojis', 'Profile Image', 'Tweet Link', 'Tweet ID', 'Comments', 'Retweets', 'Likes'], inplace=True)

In [21]:
combined_dataset.to_csv('combined_dataset.csv', index=False, encoding='utf-8')

In [22]:
combined_dataset.sample(3)

,Name,Handle,Timestamp,Content
0,T.C. Cumhurbaşkanlığı,@tcbestepe,2024-01-26T15:31:15.000Z,"Cumhurbaşkanımız , NATO Genel Sekreteri Jens Stoltenberg ile bir telefon görüşmesi gerçekleştirdi.\n\nGörüşmede, küresel ve bölgesel meseleler ve Türkiye'nin İsveç'in NATO üyeliğine verdiği onay ele alındı."
1,Hakan Fidan,@HakanFidan,2024-01-12T19:48:13.000Z,"Şehadete eren kahramanlarımıza Allah’tan rahmet; kıymetli ailelerine, yakınlarına ve aziz milletimize başsağlığı ve sabır diliyorum. Şehitlerimizin mekanları cennet, makamları âli olsun. \n\nYaralanan askerlerimizin bir an önce sağlıklarına kavuşmasını Cenab-ı Allah’tan niyaz…"
4,Alparslan Bayraktar,@aBayraktar1,2024-01-24T17:09:39.000Z,Cumhurbaşkanımız Sayın ve İran Cumhurbaşkanı Sayın İbrahim Reisi’nin katılımı ile Cumhurbaşkanlığı Külliyesi’nde düzenlenen Türkiye-İran Yüksek Düzeyli İş Birliği Konseyi Sekizinci Toplantısı’na iştirak ettik. \n\nToplantı sonrasında Bakanlığımız ile İran Petrol…


In [2]:
import pandas as pd

pd.read_csv('combined_dataset.csv')

,Name,Handle,Timestamp,Content
0,Alparslan Bayraktar,@aBayraktar1,2024-01-25T12:00:16.000Z,Bugün itibarıyla lisanssız yenilenebilir yatır...
1,Alparslan Bayraktar,@aBayraktar1,2024-01-25T12:00:16.000Z,Bugün itibarıyla lisanssız yenilenebilir yatır...
2,Alparslan Bayraktar,@aBayraktar1,2024-01-24T17:09:39.000Z,Cumhurbaşkanımız Sayın ve İran Cumhurbaşkanı ...
3,Alparslan Bayraktar,@aBayraktar1,2024-01-24T11:43:37.000Z,Cumhurbaşkanımız ve Genel Başkanımız Sayın ‘ın...
4,Alparslan Bayraktar,@aBayraktar1,2024-01-24T09:59:40.000Z,Hollanda Büyükelçisi Sayın Joep Wijnands’ı Bak...
...,...,...,...,...
226,Cevdet Yılmaz,@_cevdetyilmaz,2024-01-26T15:21:17.000Z,Uluslararası Adalet Divanı’nın bugün verdiği k...
227,Cevdet Yılmaz,@_cevdetyilmaz,2024-01-26T08:26:58.000Z,Anadolu Ajansı muhabirleri ve foto muhabirleri...
228,Cevdet Yılmaz,@_cevdetyilmaz,2024-01-26T07:26:56.000Z,Kastamonu’da bir yolcu otobüsünün devrilmesi s...
229,Cevdet Yılmaz,@_cevdetyilmaz,2024-01-25T13:22:52.000Z,#Malatya Battalgazi’de meydana gelen depremden...


In [24]:
combined_dataset['Handle'].value_counts()

Handle
@RTErdogan          10
@drfahrettinkoca    10
@Merkez_Bankasi     10
@mehmetozhaseki     10
@NumanKurtulmus     10
@csgbakanligi       10
@a_uraloglu         10
@yilmaztunc          9
@tcmeb               9
@ibrahimyumakli      9
@hafizegayeerkan     8
@HakanFidan          8
@tcbestepe           8
@MehmetNuriErsoy     8
@Yusuf__Tekin        8
@tcsavunma           8
@MahinurOzdemir      7
@mfatihkacir         7
@_cevdetyilmaz       7
@isikhanvedat        6
@TCSanayi            6
@TC_Disisleri        6
@TCKulturTurizm      6
@aBayraktar1         5
@omerbolatTR         5
@adalet_bakanlik     5
@tcailesosyal        4
@memetsimsek         4
@TCEnerji            3
@UABakanligi         3
@gencliksporbak      3
@TC_icisleri         2
@OA_BAK              2
@saglikbakanligi     1
@TCTarim             1
@HMBakanligi         1
@ticaret             1
@csbgovtr            1
Name: count, dtype: int64

# Save Scraped Tweets in a CSV

In [ ]:
scraper.save_to_csv()

In [ ]:
scraper.driver.close()